In [38]:
%load_ext autoreload
%autoreload 2
import source as sc
from sig import Signal
from strategy import Strategy

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 获取历史K线

In [3]:
df = sc.get_ts_stock_data('SZ000001', '20210101')

## 定义交易信号

#### 买入

In [4]:
class LongSig(Signal):
    def evaluate(self):
        """昨天涨就买"""
        if self._kline['change'].iloc[0] > 0:
            return True
        else:
            return False

#### 止盈

In [5]:
class TpSig(Signal):
    def evaluate(self):
        """昨天跌就卖"""
        if self._kline['change'].iloc[0] < 0:
            return True
        else:
            return False

#### 止损

In [6]:
class LsSig(Signal):
    def evaluate(self):
        """昨天跌就卖"""
        if self._kline['change'].iloc[0] < 0:
            return True
        else:
            return False

## 指定策略

In [64]:
s = Strategy(LongSig, TpSig)

## 回测结果

In [65]:
def back_test(kline, strategy, start_date=None, end_date=None):
    """
    回测函数
    """
    trade_ls = []
    
    if start_date is not None:
        start_idx = kline.loc[kline['trade_date'] == start_date].index[0]
    else:
        start_idx = len(kline) - 1
        
    if end_date is not None:
        end_idx = kline.loc[kline['trade_date'] == end_date].index[0]
    else:
        end_idx = 0
    
    for i in range(start_idx-1, end_idx-1, -1):
        date = kline['trade_date'].iloc[i] # 第n天
        today = kline.iloc[i]
        hist = kline[i+1:] # 前n-1天的K线
        
        # 设置加仓/止盈/止损
        strategy.open_setting(today['open'], 1)
        strategy.take_profit_setting(today['open'])
        
        trade = {'date': date}
        trade.update(strategy.run(hist))
        trade.update({'date': date})
        trade_ls.append(trade)
    
    return trade_ls

In [66]:
res = back_test(df, s)

In [67]:
res_df = pd.DataFrame(res).dropna()
res_df

,date,operation,price,lot,position
2,20210107,buy,19.52,1.0,1.0
3,20210108,buy,19.90,1.0,2.0
4,20210111,take_profit,20.00,-2.0,0.0
5,20210112,buy,20.39,1.0,1.0
6,20210113,buy,21.00,1.0,2.0
...,...,...,...,...,...
167,20210909,take_profit,19.11,-2.0,0.0
169,20210913,buy,20.36,1.0,1.0
170,20210914,take_profit,20.18,-1.0,0.0
172,20210916,buy,19.30,1.0,1.0


In [68]:
overall_profit = (-1 * res_df['price'] * res_df['lot']).sum()
overall_profit

-0.6299999999999706